In [1]:
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
import requests
import os
import zipfile
import arcpy
import seaborn as sns

In [6]:
#set important file paths
gdb_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb"
proj_folder = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj"

#add a Hennepin County Boundary layer for context/other functions later
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps("Map")[0]
map_obj.addDataFromPath("https://gis.hennepin.us/arcgis/rest/services/HennepinData/BOUNDARIES/MapServer/1")

In [7]:
#set urls for download

streets_url = "https://hub.arcgis.com/api/v3/datasets/2c68a692df4b4e47af378a00452d85b0_0/downloads/data?format=shp&spatialRefId=26915&where=1=1"
bike_url = "https://hub.arcgis.com/api/v3/datasets/938c8598f0a0439a8d612bb7894a05cc_1/downloads/data?format=shp&spatialRefId=26915&where=1=1"

streets_zip_path = fr"{proj_folder}\Hennepin_County_Street_Centerlines.zip"
bike_zip_path = fr"{proj_folder}\Hennepin_County_Bike_and_Pedestrian_System.zip"

streets_path = fr"{proj_folder}\Hennepin_County_Street_Centerlines"
bike_path = fr"{proj_folder}\Hennepin_County_Bike_and_Pedestrian_System"

#request downloads
streets_res = requests.get(streets_url)
bike_res = requests.get(bike_url)

#save to file
with open (streets_zip_path, "wb") as f:
    f.write(streets_res.content)
    
with open (bike_zip_path, "wb") as f:
    f.write(bike_res.content)

In [8]:
#unzip files
with zipfile.ZipFile(streets_zip_path, "r") as zip_ref:
    zip_ref.extractall(streets_path)
    
with zipfile.ZipFile(bike_zip_path, "r") as zip_ref:
    zip_ref.extractall(bike_path)

In [16]:
#create random points that will form the basis of travel time and network distance analysis
arcpy.management.CreateRandomPoints(
    out_path=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb",
    out_name="bike_points_100",
    constraining_feature_class="Hennepin County Boundary",
    constraining_extent="DEFAULT",
    number_of_points_or_field=100,
    minimum_allowed_distance="0 Meters",
    create_multipoint_output="POINT",
    multipoint_size=0
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\bike_points_100'>

In [18]:
#create a line feature class of just local streets - this will build into the low quality network

arcpy.conversion.ExportFeatures(
    in_features="Hennepin_County_Street_Centerlines",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Hennepin_Streets_Local",
    where_clause="ROUTE_SYS <> '01' And ROUTE_SYS <> '02' And ROUTE_SYS <> '03' And ROUTE_SYS <> '04' And ROUTE_SYS <> '07' And ROUTE_SYS <> '16' And ROUTE_SYS <> '22' And ROUTE_SYS <> '23' And ROUTE_SYS <> '41' And ST_NAME NOT LIKE '%Ramp%'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Hennepin_Streets_Local'>

In [19]:
#narrow bike and ped system to just bike system

arcpy.conversion.ExportFeatures(
    in_features="Hennepin_County_Bike_and_Pedestrian_System",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Hennepin_Bike_Sys",
    where_clause="FACILITY = 'Bike' Or FACILITY = 'Shared'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)

#This is new basis for bike network - all other version will build from this. This includes planned bike routes.

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Hennepin_Bike_Sys'>

In [3]:
'''
Will create three versions of bike network :
Only high quality bike facilities (High comfort network)
All bike-only facilities (Medium comfort network)
Local streets + all bike only facilities (low comfort network)
'''
#Create all bike facilities that are open (this eliminates planned bikeways which are included in the dataset):
arcpy.conversion.ExportFeatures(
    in_features="Hennepin_Bike_Sys",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Hennepin_Bike_Sys_All",
    where_clause="STATUS = 'Open'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)



<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Hennepin_Bike_All'>

In [5]:
'''
Create high quality system - this is ONLY facilities that provide a manner of protection to riders
Bike bridges/tunnels, bike-only trails, contraflow bike lanes, Cycle track, Multi-use trail, Separated use trail, 
on-street protected lane
'''
arcpy.conversion.ExportFeatures(
    in_features="Hennepin_Bike_Sys_All",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Henn_High_Qual_Bike_Sys",
    where_clause="SPEC_TYPE = 'Bike/Pedestrian Bridge' Or SPEC_TYPE = 'Bike/Pedestrian Tunnel' Or SPEC_TYPE = 'Bike-Only Trail' Or SPEC_TYPE = 'Contraflow Bike Lane' Or SPEC_TYPE = 'Cycle Track' Or SPEC_TYPE = 'Multi-Use Trail' Or SPEC_TYPE = 'On-Street Protected Bike Lane' Or SPEC_TYPE = 'Separated-Use Trail'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Henn_High_Qual_Cart'>

In [23]:
#Due to data quality issues, will need to create a new version of the bike network that fills in gaps. Use Inegrate tool

with arcpy.EnvManager(XYTolerance="20 Meters"):
    arcpy.management.Integrate(
        in_features="Hennepin_Bike_Sys_All #",
        cluster_tolerance=None
    )
    
with arcpy.EnvManager(XYTolerance="20 Meters"):
    arcpy.management.Integrate(
        in_features="Henn_High_Qual_Bike_Sys #",
        cluster_tolerance=None
    )

In [24]:
#create duplicate of current bike system to go into low-quality network
arcpy.conversion.ExportFeatures(
    in_features="Hennepin_Bike_Sys_All",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Henn_Low_Qual_Bike",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Henn_Low_Qual_Bike'>

In [25]:
#snap local streets to duplicate network to fill in minor gaps, ensure networks actually connect when they go into network

arcpy.edit.Snap(
    in_features="Henn_Low_Qual_Bike",
    snap_environment="Street_Centerlines_Local END '25 Meters'"
)

<Result 'Henn_Low_Qual_Bike'>

In [26]:
#create feature dataset

arcpy.management.CreateFeatureDataset(
    out_dataset_path=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb",
    out_name="Bike_analysis",
    spatial_reference='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
)

#need to move bike sysyem feature classes into here by hand (probably)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Bike_analysis'>

In [ ]:
#Next steps are to use arcpy to initiate network dataset, set network properties, and conduct basics of network analysis to compare different networks

## Move the input data into the feature dataset by hand in the Catalog pane now

In [30]:
#create three Network datasets

arcpy.na.CreateNetworkDataset(
    feature_dataset=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Bike_analysis",
    out_name="Henn_high_qual",
    source_feature_class_names="Henn_High_Qual_Bike_Sys",
    elevation_model="NO_ELEVATION"
)

arcpy.na.CreateNetworkDataset(
    feature_dataset=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Bike_analysis",
    out_name="Henn_med_qual",
    source_feature_class_names="Hennepin_Bike_Sys_All",
    elevation_model="NO_ELEVATION"
)

arcpy.na.CreateNetworkDataset(
    feature_dataset=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Bike_analysis",
    out_name="Henn_low_qual",
    source_feature_class_names="Henn_Low_Qual_Bike;Hennepin_Streets_Local",
    elevation_model="NO_ELEVATION"
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Bike_analysis\\Henn_low_qual'>

#### Manually remove network datasets from Contents pane

#### Set Network Properties (right click on each dataset in Catalog pane) - ensure sources and connectivity policies are correct - use any vertices for connections

#### Also set Costs (!ShapeSTLen!) field is meters - can convert to whatever units from there. Time is optional but can be set too.

In [31]:
#Build all three networks

arcpy.na.BuildNetwork(
    in_network_dataset="Henn_High_Qual_Bike_Sys",
    force_full_build="NO_FORCE_FULL_BUILD"
)

arcpy.na.BuildNetwork(
    in_network_dataset="Henn_med_qual",
    force_full_build="NO_FORCE_FULL_BUILD"
)

arcpy.na.BuildNetwork(
    in_network_dataset="Henn_low_qual",
    force_full_build="NO_FORCE_FULL_BUILD"
)

#Ensure that all three are working correctly and don't have any dirty areas

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 030033: Parameter does not contain a network dataset data element.
Failed to execute (BuildNetwork).


In [ ]:
'''
# This is probably easier to do by hand, but can be done by code if everything works right.
# Need to update the input parameters here before running it for real.

arcpy.na.MakeODCostMatrixAnalysisLayer(
    network_data_source=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Bike_analysis\Henn_low_qual",
    layer_name="OD Cost Matrix",
    travel_mode="",
    cutoff=None,
    number_of_destinations_to_find=None,
    time_of_day=None,
    time_zone="LOCAL_TIME_AT_LOCATIONS",
    line_shape="STRAIGHT_LINES",
    accumulate_attributes=None,
    ignore_invalid_locations="SKIP"
)

# Add all points as both origins and destinations - we want to know the distance and time from each point to every other point
# Snap offset set to 100 meters to ensure we can get ONTO the network from each point, search tolerance  = 1km

arcpy.na.AddLocations(
    in_network_analysis_layer="OD Cost Matrix",
    sub_layer="Origins",
    in_table="bike_points_100",
    field_mappings="Name OID #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Length # #",
    search_tolerance="1000 Meters",
    sort_field=None,
    search_criteria="Hennepin_Streets_Local SHAPE;Henn_Low_Qual_Bike SHAPE;Henn_low_qual_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="CLEAR",
    snap_to_position_along_network="SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

arcpy.na.AddLocations(
    in_network_analysis_layer="OD Cost Matrix - Low",
    sub_layer="Destinations",
    in_table="bike_points_100",
    field_mappings="Name OID #;CurbApproach # 0",
    search_tolerance="1000 Meters",
    sort_field=None,
    search_criteria="Hennepin_Streets_Local SHAPE;Henn_Low_Qual_Bike SHAPE;Henn_low_qual_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="CLEAR",
    snap_to_position_along_network="SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

#Solve to get table of all distances and times

arcpy.na.Solve(
    in_network_analysis_layer="OD Cost Matrix - Low",
    ignore_invalids="SKIP",
    terminate_on_solve_error="TERMINATE",
    simplification_tolerance=None,
    overrides=""
)

#Next do med quality network
arcpy.na.MakeODCostMatrixAnalysisLayer(
    network_data_source=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Bike_analysis\Henn_med_qual",
    layer_name="OD Cost Matrix",
    travel_mode="",
    cutoff=None,
    number_of_destinations_to_find=None,
    time_of_day=None,
    time_zone="LOCAL_TIME_AT_LOCATIONS",
    line_shape="STRAIGHT_LINES",
    accumulate_attributes=None,
    ignore_invalid_locations="SKIP"
)

arcpy.na.AddLocations(
    in_network_analysis_layer="OD Cost Matrix - Med",
    sub_layer="Origins",
    in_table="bike_points_100",
    field_mappings="Name OID #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Length # #",
    search_tolerance="1000 Meters",
    sort_field=None,
    search_criteria="Hennepin_Bike_Sys_All SHAPE;Henn_med_qual_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="APPEND",
    snap_to_position_along_network="SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

arcpy.na.AddLocations(
    in_network_analysis_layer="OD Cost Matrix - Med",
    sub_layer="Destinations",
    in_table="bike_points_100",
    field_mappings="Name OID #;CurbApproach # 0",
    search_tolerance="1000 Meters",
    sort_field=None,
    search_criteria="Hennepin_Bike_Sys_All SHAPE;Henn_med_qual_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="CLEAR",
    snap_to_position_along_network="SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

arcpy.na.Solve(
    in_network_analysis_layer="OD Cost Matrix - Med",
    ignore_invalids="SKIP",
    terminate_on_solve_error="TERMINATE",
    simplification_tolerance=None,
    overrides=""
)

#Last, make high quality network
arcpy.na.MakeODCostMatrixAnalysisLayer(
    network_data_source=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Bike_analysis\Henn_high_qual",
    layer_name="OD Cost Matrix",
    travel_mode="",
    cutoff=None,
    number_of_destinations_to_find=None,
    time_of_day=None,
    time_zone="LOCAL_TIME_AT_LOCATIONS",
    line_shape="STRAIGHT_LINES",
    accumulate_attributes=None,
    ignore_invalid_locations="SKIP"
)

arcpy.na.AddLocations(
    in_network_analysis_layer="OD Cost Matrix",
    sub_layer="Origins",
    in_table="bike_points_100",
    field_mappings="Name OID #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Length # #",
    search_tolerance="1000 Meters",
    sort_field=None,
    search_criteria="Henn_High_Qual_Bike_Sys SHAPE;Henn_high_qual_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="CLEAR",
    snap_to_position_along_network="SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

arcpy.na.AddLocations(
    in_network_analysis_layer="OD Cost Matrix",
    sub_layer="Destinations",
    in_table="bike_points_100",
    field_mappings="Name OID #;CurbApproach # 0",
    search_tolerance="1000 Meters",
    sort_field=None,
    search_criteria="Henn_High_Qual_Bike_Sys SHAPE;Henn_high_qual_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="CLEAR",
    snap_to_position_along_network="SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

arcpy.na.Solve(
    in_network_analysis_layer="OD Cost Matrix",
    ignore_invalids="SKIP",
    terminate_on_solve_error="TERMINATE",
    simplification_tolerance=None,
    overrides=""
)
'''

### When adding Origins/Destinations - change search tolerance to 1000m 
#### This will limit the amount of points that are connected to the network, but the default setting of 5000m is too far. What this means is that we are giving our "cyclists" 1km to get to the network.

In [ ]:
#export the tables for easier data manipulation
#First - rename both OD Cost Matrix Analysis Layer and Lines sublayer with correct label for network quality

arcpy.conversion.ExportTable(
    in_table=r"OD Cost Matrix - Low\Lines - Low",
    out_table=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Lines_low.csv",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=r'Name "Name" true true true 1024 Text 0 0,First,#,OD Cost Matrix - Low\Lines - Low,Name,0,1023;OriginID "OriginID" true true true 4 Long 0 0,First,#,OD Cost Matrix - Low\Lines - Low,OriginID,-1,-1;DestinationID "DestinationID" true true true 4 Long 0 0,First,#,OD Cost Matrix - Low\Lines - Low,DestinationID,-1,-1;DestinationRank "DestinationRank" true true true 4 Long 0 0,First,#,OD Cost Matrix - Low\Lines - Low,DestinationRank,-1,-1;Total_Length "Total_Length" true true true 8 Double 0 0,First,#,OD Cost Matrix - Low\Lines - Low,Total_Length,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,OD Cost Matrix - Low\Lines - Low,Shape_Length,-1,-1',
    sort_field=None
)

arcpy.conversion.ExportTable(
    in_table=r"OD Cost Matrix - Med\Lines - Med",
    out_table=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Lines_med.csv",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=r'Name "Name" true true true 1024 Text 0 0,First,#,OD Cost Matrix - Med\Lines - Med,Name,0,1023;OriginID "OriginID" true true true 4 Long 0 0,First,#,OD Cost Matrix - Med\Lines - Med,OriginID,-1,-1;DestinationID "DestinationID" true true true 4 Long 0 0,First,#,OD Cost Matrix - Med\Lines - Med,DestinationID,-1,-1;DestinationRank "DestinationRank" true true true 4 Long 0 0,First,#,OD Cost Matrix - Med\Lines - Med,DestinationRank,-1,-1;Total_Length "Total_Length" true true true 8 Double 0 0,First,#,OD Cost Matrix - Med\Lines - Med,Total_Length,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,OD Cost Matrix - Med\Lines - Med,Shape_Length,-1,-1',
    sort_field=None
)

arcpy.conversion.ExportTable(
    in_table=r"OD Cost Matrix - High\Lines - High",
    out_table=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Lines_high.csv",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=r'Name "Name" true true true 1024 Text 0 0,First,#,OD Cost Matrix - High\Lines - High,Name,0,1023;OriginID "OriginID" true true true 4 Long 0 0,First,#,OD Cost Matrix - High\Lines - High,OriginID,-1,-1;DestinationID "DestinationID" true true true 4 Long 0 0,First,#,OD Cost Matrix - High\Lines - High,DestinationID,-1,-1;DestinationRank "DestinationRank" true true true 4 Long 0 0,First,#,OD Cost Matrix - High\Lines - High,DestinationRank,-1,-1;Total_Length "Total_Length" true true true 8 Double 0 0,First,#,OD Cost Matrix - High\Lines - High,Total_Length,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,OD Cost Matrix - High\Lines - High,Shape_Length,-1,-1',
    sort_field=None
)

In [ ]:
#Next steps - calculate times for three different speed scenarios
# Calculate number of points reachable in 30 minutes or less from each point in each scenario
#Get score for all nine scenarios - visualize (for cartography and slides)

#Combine tables into one for seaborn manipulation and visualization 

#See next notebook for this work.